# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 28 2022 1:51PM,243004,10,SNAP15234059,"Snap Medical Industries, LLC",Released
1,Jul 28 2022 1:47PM,243003,16,8334358,Sartorius Bioprocess Solutions,Released
2,Jul 28 2022 1:41PM,243002,10,8334933,"Citieffe, Inc.",Released
3,Jul 28 2022 1:37PM,243001,10,8334940,Yusen – 3D Matrix,Released
4,Jul 28 2022 12:55PM,242996,10,Enova-10381,Emerginnova,Released
5,Jul 28 2022 12:55PM,242996,10,Enova-10382,Emerginnova,Released
6,Jul 28 2022 12:50PM,242911,19,GP07292022,"Granules Pharmaceuticals, Inc.",Executing
7,Jul 28 2022 12:43PM,242992,10,MTR-22-0240,"Senseonics, Incorporated",Released
8,Jul 28 2022 12:40PM,242991,10,0085914528,ISDIN Corporation,Released
9,Jul 28 2022 12:40PM,242991,10,0085914272,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,242996,Released,2
30,243001,Released,1
31,243002,Released,1
32,243003,Released,1
33,243004,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242996,NaN,NaN,2.0
243001,NaN,NaN,1.0
243002,NaN,NaN,1.0
243003,NaN,NaN,1.0
243004,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242886,7.0,0.0,13.0
242889,0.0,2.0,1.0
242900,0.0,0.0,1.0
242901,1.0,0.0,5.0
242905,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242886,7,0,13
242889,0,2,1
242900,0,0,1
242901,1,0,5
242905,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242886,7,0,13
1,242889,0,2,1
2,242900,0,0,1
3,242901,1,0,5
4,242905,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242886,7,,13
1,242889,,2,1
2,242900,,,1
3,242901,1,,5
4,242905,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 28 2022 1:51PM,243004,10,"Snap Medical Industries, LLC"
1,Jul 28 2022 1:47PM,243003,16,Sartorius Bioprocess Solutions
2,Jul 28 2022 1:41PM,243002,10,"Citieffe, Inc."
3,Jul 28 2022 1:37PM,243001,10,Yusen – 3D Matrix
4,Jul 28 2022 12:55PM,242996,10,Emerginnova
6,Jul 28 2022 12:50PM,242911,19,"Granules Pharmaceuticals, Inc."
7,Jul 28 2022 12:43PM,242992,10,"Senseonics, Incorporated"
8,Jul 28 2022 12:40PM,242991,10,ISDIN Corporation
14,Jul 28 2022 12:39PM,242989,10,ISDIN Corporation
51,Jul 28 2022 12:39PM,242990,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 28 2022 1:51PM,243004,10,"Snap Medical Industries, LLC",,,1
1,Jul 28 2022 1:47PM,243003,16,Sartorius Bioprocess Solutions,,,1
2,Jul 28 2022 1:41PM,243002,10,"Citieffe, Inc.",,,1
3,Jul 28 2022 1:37PM,243001,10,Yusen – 3D Matrix,,,1
4,Jul 28 2022 12:55PM,242996,10,Emerginnova,,,2
5,Jul 28 2022 12:50PM,242911,19,"Granules Pharmaceuticals, Inc.",,1,
6,Jul 28 2022 12:43PM,242992,10,"Senseonics, Incorporated",,,1
7,Jul 28 2022 12:40PM,242991,10,ISDIN Corporation,,,6
8,Jul 28 2022 12:39PM,242989,10,ISDIN Corporation,,,37
9,Jul 28 2022 12:39PM,242990,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 28 2022 1:51PM,243004,10,"Snap Medical Industries, LLC",1,,
1,Jul 28 2022 1:47PM,243003,16,Sartorius Bioprocess Solutions,1,,
2,Jul 28 2022 1:41PM,243002,10,"Citieffe, Inc.",1,,
3,Jul 28 2022 1:37PM,243001,10,Yusen – 3D Matrix,1,,
4,Jul 28 2022 12:55PM,242996,10,Emerginnova,2,,
5,Jul 28 2022 12:50PM,242911,19,"Granules Pharmaceuticals, Inc.",,1,
6,Jul 28 2022 12:43PM,242992,10,"Senseonics, Incorporated",1,,
7,Jul 28 2022 12:40PM,242991,10,ISDIN Corporation,6,,
8,Jul 28 2022 12:39PM,242989,10,ISDIN Corporation,37,,
9,Jul 28 2022 12:39PM,242990,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 28 2022 1:51PM,243004,10,"Snap Medical Industries, LLC",1,,
1,Jul 28 2022 1:47PM,243003,16,Sartorius Bioprocess Solutions,1,,
2,Jul 28 2022 1:41PM,243002,10,"Citieffe, Inc.",1,,
3,Jul 28 2022 1:37PM,243001,10,Yusen – 3D Matrix,1,,
4,Jul 28 2022 12:55PM,242996,10,Emerginnova,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 28 2022 1:51PM,243004,10,"Snap Medical Industries, LLC",1.0,NaN,NaN
1,Jul 28 2022 1:47PM,243003,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
2,Jul 28 2022 1:41PM,243002,10,"Citieffe, Inc.",1.0,NaN,NaN
3,Jul 28 2022 1:37PM,243001,10,Yusen – 3D Matrix,1.0,NaN,NaN
4,Jul 28 2022 12:55PM,242996,10,Emerginnova,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 28 2022 1:51PM,243004,10,"Snap Medical Industries, LLC",1.0,0.0,0.0
1,Jul 28 2022 1:47PM,243003,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
2,Jul 28 2022 1:41PM,243002,10,"Citieffe, Inc.",1.0,0.0,0.0
3,Jul 28 2022 1:37PM,243001,10,Yusen – 3D Matrix,1.0,0.0,0.0
4,Jul 28 2022 12:55PM,242996,10,Emerginnova,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2429811,61.0,1.0,0.0
16,971842,4.0,0.0,0.0
19,1214545,12.0,27.0,24.0
20,971640,45.0,10.0,8.0
21,242990,1.0,0.0,0.0
22,728913,1.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2429811,61.0,1.0,0.0
1,16,971842,4.0,0.0,0.0
2,19,1214545,12.0,27.0,24.0
3,20,971640,45.0,10.0,8.0
4,21,242990,1.0,0.0,0.0
5,22,728913,1.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,61.0,1.0,0.0
1,16,4.0,0.0,0.0
2,19,12.0,27.0,24.0
3,20,45.0,10.0,8.0
4,21,1.0,0.0,0.0
5,22,1.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,61.0
1,16,Released,4.0
2,19,Released,12.0
3,20,Released,45.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,21,22
Status,,,,,,
Completed,0.0,0.0,24.0,8.0,0.0,0.0
Executing,1.0,0.0,27.0,10.0,0.0,2.0
Released,61.0,4.0,12.0,45.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,21,22
0,Completed,0.0,0.0,24.0,8.0,0.0,0.0
1,Executing,1.0,0.0,27.0,10.0,0.0,2.0
2,Released,61.0,4.0,12.0,45.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,21,22
0,Completed,0.0,0.0,24.0,8.0,0.0,0.0
1,Executing,1.0,0.0,27.0,10.0,0.0,2.0
2,Released,61.0,4.0,12.0,45.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()